In [1]:
# !pip install multiprocess # 이거 절대로 쓰면 안된다
                            # 써봤다가 노트북 나갈 뻔했다

In [ ]:
import multiprocessing as mp
import numpy as np
import os

'''
    multiprocessing 모듈은 윈도우와 유닉스/리눅스에서 다르게 작동한다
    <윈도우>
    윈도우의 경우 fork() system call이 없어 새로운 프로세스를 시작(start)할 경우, 새로운 파이썬 인터프리터를 시작함과 동시에 전체 프로그램을 처음부터 다시 실행한다
    따라서 'if __name__ == "__main__":'을 명시하지 않으면 프로세스의 생성이 무한히 반복된다
    처음 파이썬 프로그램을 시작하여 생성된 첫번째 프로세스는 __name__에 "__main__"이 들어있지만 두번째 프로세스부터는 "__mp_main__"이 들어있다
    따라서 'if __name__ == "__main__":'을 명시하여 코드를 작성하면, 새로이 시작된 프로세스가 프로그램의 처음부터 다시 실행할 때 if문 안쪽의 (여기서는)mp.Pool(8)을 실행하지 않아 프로세스가 무한히 생성되는 것을 막는다
    
    <유닉스/리눅스>
    유닉스/리눅스의 경우 fork() system call이 존재하기 때문에 새로운 프로세스 생성 명령이 실행되면 현재 프로세스를 부모로 하는 자식 프로세스를 생성한다
    생성된 자식 프로세스는 프로그램의 처음부터 다시 실행하는 것이 아닌 mp.Pool(8)와 같은 프로세스 생성 명령 바로 다음 명령부터 실행하기 때문에 프로세스가 무한정 생성되지 않는다

    <쥬피터에서 멀티-프로세싱이 안되는 이유>
    "윈도우"에서 파이썬 파일(.py)을 사용한 일반적인 멀티-프로세싱 실행 과정은 아래와 같다
    1. 새로운 프로세스를 생성 명령을 호출하면 파이썬은 해당 명령을 호출한 (실제로 그런 것은 아니지만 이하 '부모 프로세스'라고 칭한다)main module(.py 파일 혹은 전체 파이썬 코드가 저장된 파일)과 배정된 작업을 처리하기 위한 인자(데이터)를 pickle(serialize, 직렬화)하여 새로운 프로세스로 보낸다
    2. 새로운 프로세스(이하 '자식 프로세스'라 칭함)는 새로운 파이썬 인터프리터를 실행하고 넘겨받은 main module(.py 파일)을 다시 import 한다(즉, .py 파일에 적힌 코드를 처음부터 다시 실행한다는 것이다)
    3. 이후, 자식 프로세스는 전달받은 인자를 unpickle(deserialize, 비직렬화)하고, 이를 입력 인자(데이터)로 하여 배정된 작업 함수를 호출한다
    4. 자식 프로세스가 main modeule(.py 파일)의 실행을 완료하면, 배정된 작업(mp_func())에 해당하는 함수의 반환된 결과를 pickle(serialize, 직렬화)하여 부모 프로세스에게 보낸다
    5. 부모 프로세스는 전달 받은 결과물을 unpickle(deserialize, 비직렬화)하여 취합하고, 나머지 파이썬 코드를 계속 실행한다
    
    헌데 쥬피터 노트북은(랩도 마찬가지) 일반적인 파이썬 스크립트가 아닌, 백그라운드에서 상호작용 가능한 파이썬 커널을 실행하고 있는 웹 애플리케이션이다
    이는 main module(여기서는 .ipynb 파일)이 일반적인 파이썬 스크립트가 아니라 파이썬 객체임을 의미한다
    이러한 파이썬 객체는 pickle(serialize, 직렬화)할 수 없고 따라서 이를 새로운 프로세스에 보내줄 수 없다
    따라서 새로운 프로세스가 작동할 수 없으며, 이것이 쥬피터 상에서 (if문을 둘째치고서라도)멀티-프로세싱이 작동하지 않는 이유다

    <해결방법(4가지가 있지만 그 중 가장 간편한 방법)>
    멀티 프로세싱으로 처리할 작업에 해당하는 함수를 따로 파이썬 파일로 생성하고 그것을 불러온다
    위 방법이 가능한 이유는 아래와 같다
    1. 윈도우 상에서 새로운 프로세스를 생성할 때 main module을 넘겨준다는 했는데, 이는 새로운 프로세스에 배정한 작업 함수가 main module내에 작성되어있기 때문이다
    --> 따라서 작업에 해당하는 함수를 .py 파일로 별도로 생성하게 되면 pickle이 가능해져 main module을 새로운 프로세스에 전달해 줄 수 있게 된다
    2. 작업을 별도의 .py 파일로 생성할 때 "작업에 해당하는 함수만 생성"작성하였고 "프로세스 생성문은 작성하지 않았기 때문에" 무한 프로세스 생성 루프에 빠지지 않는다(빠질 수 없다)
    
    <참고>
    https://bobswinkels.com/posts/multiprocessing-python-windows-jupyter/
    how to use pool.map in jupyter notebook : 검색 문구
'''

def square(x):
    return np.square(x)

# 아래의 if문을 사용하지 않으면 무한 오류에 빠지게 된다
# 쥬피터에서는 아래의 if문을 사용해도 멀티 프로세싱이 수행되지 않는다
if __name__ == "__main__":
    x = np.arange(64) # 0 ~ 63까지의 정수 생성
    print(x, type(x), x.dtype)
    print("Current Host System cpu core number :", mp.cpu_count()) # 현재 호스트 시스템의 cpu(코어) 개수를 반환한다
                                                               # 그러나 이것이 현재 프로세스가 사용 가능한 cpu(코어)의 개수를 의미하지는 않는다
    pool = mp.Pool(8) # 8개의 프로세스를 가지는 프로세스 풀을 생성한다
    squared = pool.map(square, [x[8*i:8*i+8] for i in range(8)]) # 총 64개의 숫자를 8개씩 분할하여 각 프로세스에 배정한다
    print(squared)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] <class 'numpy.ndarray'> int32
Current Host System cpu core number : 16


In [8]:
import numpy as np
import multiprocessing as mp
import os
from lib.worker import square

x = np.arange(64) # 0 ~ 63까지의 정수 생성
print(x, type(x), x.dtype)
print("Current Host System cpu core number :", mp.cpu_count()) # 현재 호스트 시스템의 cpu(코어) 개수를 반환한다
                                                               # 그러나 이것이 현재 프로세스가 사용 가능한 cpu(코어)의 개수를 의미하지는 않는다
# with 구문을 사용하지 않아도 코드 실행에 아무런 문제는 없지만,
# with을 사용하는 것이 자원 관리 측면에서 훨씬 더 좋다
with mp.Pool(8) as pool:
    squared = pool.map(square, [x[8*i:8*i+8] for i in range(8)]) # 총 64개의 숫자를 8개씩 분할하여 각 프로세스에 배정한다
                                                                 # 각 프로세스가 수행햐야 되는 작업(함수)과 작업물(데이터)을 명시한다
for i in range(len(squared)):
    print(squared[i])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] <class 'numpy.ndarray'> int32
Current Host System cpu core number : 16
[ 0  1  4  9 16 25 36 49]
[ 64  81 100 121 144 169 196 225]
[256 289 324 361 400 441 484 529]
[576 625 676 729 784 841 900 961]
[1024 1089 1156 1225 1296 1369 1444 1521]
[1600 1681 1764 1849 1936 2025 2116 2209]
[2304 2401 2500 2601 2704 2809 2916 3025]
[3136 3249 3364 3481 3600 3721 3844 3969]


In [16]:
import numpy as np
import multiprocessing as mp
from lib.worker import square2

processes = [] # 프로세스 풀 생성
queue = mp.Queue() # 프로세스가 반환하는 결과물을 취합하기 위한 큐 생성
x = np.arange(64)

print("Current available Host System cpu core number :", mp.cpu_count())

# mp.Pool()은 어떤 프로세스에게 어떤 작업을 배정할지를 자동으로 결정하는 반면,
# mp.Process()는 어떤 프로세스에게 어떤 작업을 배정할지를 수동으로 지정한다
# 프로세스 작업 수행 과정은 CMD에서 확인할 수 있다
for i in range(8):
    start_index = 8 * i
    # multiprocessing에서 process는 Process 객체가 생성되고 이 객체가 start() 메서드가 호출되었을 때 소환된다(spawn)
    # Process 객체는 분리된 프로세스 내에서 실행중인 활동을 나타낸
    proc = mp.Process(target=square2, args=(i, x[start_index:start_index+8], queue)) # 프로세스 번호, 작업물, 결과물 저장 큐를 넘겨준다
    # proc.run() # 프로세스의 활동을 출력한다
               # 이 메서드를 사용하지 않으면 (쥬피터의 경우) CMD에 프로세스 활동이 출력된다
    proc.start() # 프로세스를 시작한다
    processes.append(proc) # 추후 프로세스들을 제어하기 위해 프로세스들을 추적한다

for proc in processes:
    proc.join() # 프로세스가 종료될 때까지 기다린다

for proc in processes:
    proc.terminate() # 모든 프로세스를 종료한다(자원 반환은 수행하지 않는다)
                     # 부모 프로세스를 종료하더라도 그 밑의 자식 프로세스들은 종료되지 않는다(이 경우 자식 프로세스들은 "기아"가 된다)

for proc in processes:
    proc.close()

results = []
while not queue.empty(): # 큐가 비어있지 않을 경우, 큐 안의 결과물을 반환한다
    results.append(queue.get())

for i in range(len(results)):
    print(results[i])

Current available Host System cpu core number : 16
[256 289 324 361 400 441 484 529]
[ 0  1  4  9 16 25 36 49]
[1024 1089 1156 1225 1296 1369 1444 1521]
[ 64  81 100 121 144 169 196 225]
[1600 1681 1764 1849 1936 2025 2116 2209]
[576 625 676 729 784 841 900 961]
[2304 2401 2500 2601 2704 2809 2916 3025]
[3136 3249 3364 3481 3600 3721 3844 3969]


In [4]:
import torch
from torch import nn, optim
import numpy as np
from torch.nn import functional as F
import torch.multiprocessing as mp
import gym



[]
